<a href="https://colab.research.google.com/github/DikshantBadawadagi/Encryption-Algorithms/blob/main/DNA-Encoding-Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from PIL import Image
import random

def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    array = np.clip(array, 0, 255).astype(np.uint8)
    image = Image.fromarray(array)
    image.save(output_path)

# DNA coding functions
def int_to_dna(n):
    dna_map = {0: 'A', 1: 'T', 2: 'C', 3: 'G'}
    return dna_map[n]

def dna_to_int(dna):
    dna_map = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    return dna_map[dna]

def pixel_to_dna(pixel):
    return ''.join(int_to_dna((pixel >> i) & 3) for i in (6, 4, 2, 0))

def dna_to_pixel(dna):
    return sum(dna_to_int(dna[i]) << (6 - 2*i) for i in range(4))

def dna_encode(array):
    return np.vectorize(pixel_to_dna)(array)

def dna_decode(dna_array):
    return np.vectorize(dna_to_pixel)(dna_array)

def dna_xor(dna1, dna2):
    return ''.join(int_to_dna(dna_to_int(a) ^ dna_to_int(b)) for a, b in zip(dna1, dna2))

def generate_key(shape):
    return np.random.randint(0, 256, shape, dtype=np.uint8)

def dna_encrypt(image_array, key_array):
    dna_image = dna_encode(image_array)
    dna_key = dna_encode(key_array)
    encrypted_dna = np.frompyfunc(dna_xor, 2, 1)(dna_image, dna_key)
    return dna_decode(encrypted_dna)

def dna_decrypt(encrypted_array, key_array):
    return dna_encrypt(encrypted_array, key_array)  # XOR is its own inverse

def encrypt_image_file(input_path, output_path):
    image_array = image_to_array(input_path)
    key_array = generate_key(image_array.shape)
    encrypted_array = dna_encrypt(image_array, key_array)
    array_to_image(encrypted_array, output_path)
    return key_array

def decrypt_image_file(encrypted_path, output_path, key_array):
    encrypted_array = image_to_array(encrypted_path)
    decrypted_array = dna_decrypt(encrypted_array, key_array)
    array_to_image(decrypted_array, output_path)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encryption
key_array = encrypt_image_file(input_image_path, encrypted_image_path)
print("Encryption complete.")

# Decryption
decrypt_image_file(encrypted_image_path, decrypted_image_path, key_array)
print("Decryption complete.")

# Debugging
original_image = image_to_array(input_image_path)
encrypted_image = image_to_array(encrypted_image_path)
decrypted_image = image_to_array(decrypted_image_path)

print("Original image shape:", original_image.shape)
print("Encrypted image shape:", encrypted_image.shape)
print("Decrypted image shape:", decrypted_image.shape)

print("Original image dtype:", original_image.dtype)
print("Encrypted image dtype:", encrypted_image.dtype)
print("Decrypted image dtype:", decrypted_image.dtype)

print("Original image sample:", original_image[0, 0])
print("Encrypted image sample:", encrypted_image[0, 0])
print("Decrypted image sample:", decrypted_image[0, 0])

difference = np.abs(original_image.astype(np.float64) - decrypted_image.astype(np.float64))
mean_difference = np.mean(difference)
print(f"Mean difference between original and decrypted images: {mean_difference}")

max_difference = np.max(difference)
print(f"Maximum difference between original and decrypted images: {max_difference}")

# Check if any pixel values are different
if np.array_equal(original_image, decrypted_image):
    print("The decrypted image is identical to the original image.")
else:
    print("The decrypted image differs from the original image.")
    different_pixels = np.sum(original_image != decrypted_image)
    print(f"Number of different pixels: {different_pixels}")

Encryption complete.
Decryption complete.
Original image shape: (225, 225, 3)
Encrypted image shape: (225, 225, 3)
Decrypted image shape: (225, 225, 3)
Original image dtype: uint8
Encrypted image dtype: uint8
Decrypted image dtype: uint8
Original image sample: [255 255 255]
Encrypted image sample: [185 166  88]
Decrypted image sample: [255 255 255]
Mean difference between original and decrypted images: 0.0
Maximum difference between original and decrypted images: 0.0
The decrypted image is identical to the original image.
